In [48]:
# # Install if not already installed
# !pip install sentence-transformers langchain chromadb

# from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings
# from uuid import uuid4

# # Initialize a free embedding model
# embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # Example: your documents for chatbot knowledge base
# chunks = [
#     "Hello! I am your assistant.",
#     "You can ask me questions based on my knowledge base."
# ]
# uuids = [str(uuid4()) for _ in chunks]

# # Initialize Chroma vector store
# vector_store = Chroma(embedding_function=embedding_model)

# # Add documents to the vector store
# vector_store.add_documents(documents=chunks, ids=uuids)

# print("Chatbot initialized with Hugging Face embeddings!")

AttributeError: 'str' object has no attribute 'page_content'

import the .env file

In [49]:
# from dotenv import load_dotenv
# load_dotenv()

False

configuration

In [50]:
# DATA_PATH = r"data"
# CHROMA_PATH = r"chroma_db"

initiate the model

connect to the chromadb

In [51]:
# vector_store = Chroma(
#     collection_name="example_collection",
#     embedding_function=embedding_model,  
#     persist_directory=CHROMA_PATH,
# )


Set up the vectorstore to be the retriever

In [52]:
# num_results = 5
# retriever = vector_store.as_retriever(search_kwargs={'k': num_results})

call this function for every message added to the chatbot

In [53]:
# def stream_response(message, history):
#     #print(f"Input: {message}. History: {history}\n")

#     # retrieve the relevant chunks based on the question asked
#     docs = retriever.invoke(message)

#     # add all the chunks to 'knowledge'
#     knowledge = ""
#     for doc in docs:
#         knowledge += doc.page_content+"\n\n"

In [58]:
# import gradio as gr
# from huggingface_hub import InferenceClient

# # Initialize Hugging Face client (replace with your model + token)
# import os
# client = InferenceClient(
#     "mistralai/Mistral-7B-Instruct-v0.3",
#     token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
# )

# def stream_response(message, history):
#     knowledge = "..."  # replace with retrieved knowledge
    
#     rag_prompt = f"""
#     You are an assistant which answers questions based on knowledge which is provided to you.
#     While answering, you don't use your internal knowledge,
#     but solely the information in the "The knowledge" section.
#     You don't mention anything to the user about the provided knowledge.

#     The question: {message}
#     Conversation history: {history}
#     The knowledge: {knowledge}
#     """

#     print("Prompt sent to LLM:\n", rag_prompt)

#     partial_message = ""

#     response = llm.invoke(user_input)
#     return response


# # Gradio ChatInterface
# chatbot = gr.ChatInterface(
#     fn=stream_response,
#     textbox=gr.Textbox(
#         placeholder="Send to the LLM...",
#         container=False,
#         autoscroll=True,
#         scale=7,
#     ),
# )

# chatbot.launch()



initiate the Gradio app

launch the Gradio app

In [64]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from huggingface_hub import InferenceClient
from uuid import uuid4
import gradio as gr
import os

# Load environment variables (for HuggingFace token)
from dotenv import load_dotenv
load_dotenv()

# Initialize embeddings and vector store
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
CHROMA_PATH = "chroma_db"
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embedding_model,
    persist_directory=CHROMA_PATH,
)
retriever = vector_store.as_retriever(search_kwargs={'k': 5})

# Initialize HuggingFace client
client = InferenceClient(
    "mistralai/Mistral-7B-Instruct-v0.3",
    token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)

# Chat handler
def stream_response(message, history):
    # Retrieve relevant docs
    docs = retriever.invoke(message)
    knowledge = "\n\n".join([doc.page_content for doc in docs]) or "No context found."

    # Build RAG prompt
    rag_prompt = f"""
    You are an assistant which answers questions based ONLY on the provided knowledge.
    The question: {message}
    Conversation history: {history}
    The knowledge: {knowledge}
    """

    print("Prompt sent to LLM:\n", rag_prompt)

    # Send to HuggingFace model
    def stream_response(message, history):
    # Retrieve relevant docs
        docs = retriever.invoke(message)
        knowledge = "\n\n".join([doc.page_content for doc in docs]) or "No context found."

    # Build RAG prompt
        rag_prompt = f"""
        You are an assistant which answers questions based ONLY on the provided knowledge.
        The question: {message}
        Conversation history: {history}
        The knowledge: {knowledge}
        """

        print("Prompt sent to LLM:\n", rag_prompt)

    # Use Hugging Face conversational API
        response = client.conversational(
            {
                "inputs": {
                "past_user_inputs": [h[0] for h in history],
                "generated_responses": [h[1] for h in history],
                "text": rag_prompt,
                }
            }
        )

        return response.generated_text


# Gradio ChatInterface
        chatbot = gr.ChatInterface(
            fn=stream_response,
            textbox=gr.Textbox(
                placeholder="Ask me anything...",
                container=False,
                autoscroll=True,
                scale=7,
            ),
        )

chatbot.launch()


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* To create a public link, set `share=True` in `launch()`.


Prompt sent to LLM:
 
    You are an assistant which answers questions based ONLY on the provided knowledge.
    The question: Hi please tell me something about gilki
    Conversation history: []
    The knowledge: supportive faculty and vibrant student life, filled with 
diverse clubs and activities, have made my time here 
truly unforgettable. GIKI is more than just a place 
of learning; it’s a community that fosters innovation, 
leadership, and lifelong friendships.
ABDUL REHMAN

from our professors was invaluable, guiding me through 
complex concepts and encouraging me to push the 
boundaries of my understanding.
GIKI is a melting pot of people from different cultures 
in Pakistan. This diversity not only broadened my

As I continue my journey at GIKI, I am confident that 
the skills and experiences gained here will pave the 
way for future success.”
HARIS ZEB 
(POWER)

will contribute to the advancement of Pakistan and beyond. 
Institutions like GIKI, with a legacy of three decade

Traceback (most recent call last):
  File "C:\Users\MQ\anaconda\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MQ\anaconda\Lib\site-packages\gradio\route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MQ\anaconda\Lib\site-packages\gradio\blocks.py", line 2274, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MQ\anaconda\Lib\site-packages\gradio\blocks.py", line 1779, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MQ\anaconda\Lib\site-packages\gradio\utils.py", line 876, in async_wrapper
    response = await f(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MQ\anaconda\Lib\site-packag